In [ ]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import json

# Fetch Database

url_products = "https://fakestoreapi.com/products"
response_products = requests.get(url_products).json()
products = pd.DataFrame(response_products)

url_cart = "https://fakestoreapi.com/carts"
response_cart = requests.get(url_cart).json()
carts = pd.DataFrame(response_cart)

url_users = "https://fakestoreapi.com/users"
response_users = requests.get(url_users).json()
users = pd.DataFrame(response_users)

# Database Column Edits
products.rename(columns={"id":"product_id"}, inplace=True)
products.rename(columns={"title":"product_name"}, inplace=True)
carts.rename(columns={"id":"cart_id"}, inplace=True)
carts.rename(columns={"userId":"user_id"}, inplace=True)
carts.rename(columns={"date":"order_date"}, inplace=True)
users.rename(columns={"id":"user_id"}, inplace=True)

# Products Database Edits

products["rating_rate"] = products["rating"].apply(lambda x:x["rate"])
products["rating_count"] = products["rating"].apply(lambda x:x["count"])
products.drop(columns="rating", inplace=True)

# Cart Database Edits

carts_main = carts.drop(columns="products")
carts_main = carts_main.drop(columns="__v")

cart_products = carts.explode("products")
cart_products["product_id"] = cart_products["products"].apply(lambda x:x["productId"])
cart_products["quantity"] = cart_products["products"].apply(lambda x:x["quantity"])
cart_products = cart_products[["cart_id", "product_id", "quantity"]]
cart_products.reset_index(drop=True, inplace=True)

# Users Database Edits

## View contents: print(json.dumps(users["address"][0], indent=4)) -- See dictionary horizontally
users["lat"] = users["address"].apply(lambda x:x["geolocation"]["lat"])
users["long"] = users["address"].apply(lambda x:x["geolocation"]["long"])
users["city"] = users["address"].apply(lambda x:x["city"])
users["street"] = users["address"].apply(lambda x:x["street"])
users["number"] = users["address"].apply(lambda x:x["number"])
users["zipcode"] = users["address"].apply(lambda x:x["zipcode"])
users["firstname"] = users["name"].apply(lambda x:x["firstname"])
users["lastname"] = users["name"].apply(lambda x:x["lastname"])
users = users.drop(columns=["address", "__v", "name"])

# print(users.head())                     -- Check all tables
# print(users["zipcode"].str.len().max()) -- Check lengths for VARCHAR

# Connect to Cloud SQL

DB_USER = "postgres"
DB_PASSWORD = "monkeyd"
DB_HOST = "34.150.9.71"
DB_NAME = "ecommerce-db"

engine = sqlalchemy.create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:5432/{DB_NAME}")

with engine.connect() as conn:
  conn.execute("""
    CREATE TABLE products (
      id SERIAL PRIMARY KEY,
      title VARCHAR(100),
      price NUMERIC,
      description VARCHAR(800),
      category VARCHAR(20),
      image VARCHAR(150),
      rating_rate NUMERIC,
      rating_count INT
    );

    CREATE TABLE carts_main (
      cart_id INT PRIMARY KEY,
      user_id INT,
      order_date DATE
    );

    CREATE TABLE cart_products (
      cart_id INT,
      product_id INT,
      quantity INT,
      PRIMARY KEY (cart_id, product_id)
    );

    CREATE TABLE users (
      user_id INT PRIMARY KEY,
      email VARCHAR(40),
      username VARCHAR(20),
      password VARCHAR(20),
      phone VARCHAR(14),
      lat NUMERIC,
      long NUMERIC,
      city VARCHAR(20),
      street VARCHAR(20),
      number INT,
      zipcode VARCHAR(10)
    );
  """
  )

# Insert data into Cloud SQL
df.to_sql("products", engine, if_exists="replace", index=False)
df.to_sql("carts_main", engine, if_exists="replace", index=False)
df.to_sql("cart_products", engine, if_exists="replace", index=False)
df.to_sql("users", engine, if_exists="replace", index=False)
print("Data successfully inserted into Cloud SQL.")


OperationalError: (psycopg2.OperationalError) connection to server at "34.150.9.71", port 5432 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)